## Chatgpt4o-mini

In [34]:
from openai import OpenAI, Client
import os
import pandas as pd
from tqdm.auto import tqdm
import json
from IPython.display import display, clear_output, Markdown


In [2]:
with open("doc_chunked.json", 'rt') as f_in:
    documents = json.load(f_in)

In [3]:
len(documents)

614

In [4]:
documents[0]

{'source': 'https://www.reddit.com/r/germany/wiki/autobahn_safety',
 'content': 'The Autobahn is a [network of interstate highways in Germany](https://en.m.wikipedia.org/wiki/Autobahn#/media/File%3AAutobahnen_in_Deutschland.svg) with a total length of more than 8,000 miles. [65%](https://en.wikipedia.org/wiki/Autobahn#Speed_limits) of the Autobahn has no speed limit. How safe can that be?\nVehicles traveled 147 billion miles on the Autobahn in 2015. 322 people died = 2.19 deaths per billion miles.\nIn the US, vehicles travelled 757 billion miles on interstate highways. 3,837 people died = 5.07 deaths per billion miles.\nThat means: If you drive on the interstate, your likelihood to die is 131% higher than for the same distance on the Autobahn.\n*sources:*\nStatistisches Bundesamt: [Unfallentwicklung auf deutschen Straßen 2015](https://www.destatis.de/DE/PresseService/Presse/Pressekonferenzen/2016/Unfallentwicklung_2015/Pressebroschuere_unfallentwicklung.pdf?__blob=publicationFile)\nNat

In [5]:
client = OpenAI(api_key=os.environ["IA"])

In [15]:
prompt_template = """
You're a helpful assistant. For each content provided, generate one  question that a user might ask. The question should be directly relevant and focus on the key message or detail within the content. If possible, use as fewer words as possible from the content.The questions should be complete and not too short.


HEADLINE: {headline}
CONTENT: {content}

return just question as string.

""".strip()

In [43]:
def get_prompt(doc):
    content = doc["content"]
    headline = doc["headline"]
    promt = prompt_template.format(content=content, headline=headline)
    return promt

In [17]:
def gen_q(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    res = response.choices[0].message.content
    return res

In [21]:
print(get_prompt(documents[1]))

You're a helpful assistant. For each content provided, generate one  question that a user might ask. The question should be directly relevant and focus on the key message or detail within the content. If possible, use as fewer words as possible from the content.The questions should be complete and not too short.


HEADLINE: German citizenship by descent
CONTENT: *By u/staplehill*
German citizenship is [not based on where you are born](https://en.wikipedia.org/wiki/Jus_soli). You get German citizenship either through naturalization after living in Germany for a number of years, or you inherit it from a parent. German citizenship *can* be passed down through multiple generations who live abroad and have no idea that they are actually German citizens. But one can also lose German citizenship just as unknowingly, for example by voluntarily getting the citizenship of another country (1914-today), or due to sex-discriminatory laws that in the past often allowed only the father to pass on Ger

In [22]:
documents_new= []

In [23]:
for doc in tqdm(documents):
    promt = get_prompt(doc)
    res = gen_q(promt)
    id = doc["id"]
    documents_new.append({"id":id, "question":res})

  0%|          | 0/614 [00:00<?, ?it/s]

In [24]:
documents_new[0]

{'id': '9d8370cf-a2c8-4c54-9f9c-476b9c09a933',
 'question': 'What are the safety statistics comparing the Autobahn to US interstate highways?'}

In [25]:
question_map = {q['id']: q['question'] for q in  documents_new}
test = list(map(lambda doc: {**doc, 'question': question_map.get(doc['id'], None)}, documents))
len(test)

614

In [26]:
test[0]

{'source': 'https://www.reddit.com/r/germany/wiki/autobahn_safety',
 'content': 'The Autobahn is a [network of interstate highways in Germany](https://en.m.wikipedia.org/wiki/Autobahn#/media/File%3AAutobahnen_in_Deutschland.svg) with a total length of more than 8,000 miles. [65%](https://en.wikipedia.org/wiki/Autobahn#Speed_limits) of the Autobahn has no speed limit. How safe can that be?\nVehicles traveled 147 billion miles on the Autobahn in 2015. 322 people died = 2.19 deaths per billion miles.\nIn the US, vehicles travelled 757 billion miles on interstate highways. 3,837 people died = 5.07 deaths per billion miles.\nThat means: If you drive on the interstate, your likelihood to die is 131% higher than for the same distance on the Autobahn.\n*sources:*\nStatistisches Bundesamt: [Unfallentwicklung auf deutschen Straßen 2015](https://www.destatis.de/DE/PresseService/Presse/Pressekonferenzen/2016/Unfallentwicklung_2015/Pressebroschuere_unfallentwicklung.pdf?__blob=publicationFile)\nNat

In [27]:
with open('doc_with_q_4o-mini.json', 'wt') as f_out:
    json.dump(test, f_out, indent=2)

## Llama 3.1

In [30]:
llama3 = Client(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [45]:
def generate_questions(prompt):
    response = llama3.chat.completions.create(
        model="llama3.1:70b",
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [32]:
generate_questions(documents[1])

'How do I verify if my ancestor passed down German citizenship to me despite living abroad for multiple generations?'

In [50]:
documents_new_llama = []

In [42]:
documents[0]

{'source': 'https://www.reddit.com/r/germany/wiki/autobahn_safety',
 'content': 'The Autobahn is a [network of interstate highways in Germany](https://en.m.wikipedia.org/wiki/Autobahn#/media/File%3AAutobahnen_in_Deutschland.svg) with a total length of more than 8,000 miles. [65%](https://en.wikipedia.org/wiki/Autobahn#Speed_limits) of the Autobahn has no speed limit. How safe can that be?\nVehicles traveled 147 billion miles on the Autobahn in 2015. 322 people died = 2.19 deaths per billion miles.\nIn the US, vehicles travelled 757 billion miles on interstate highways. 3,837 people died = 5.07 deaths per billion miles.\nThat means: If you drive on the interstate, your likelihood to die is 131% higher than for the same distance on the Autobahn.\n*sources:*\nStatistisches Bundesamt: [Unfallentwicklung auf deutschen Straßen 2015](https://www.destatis.de/DE/PresseService/Presse/Pressekonferenzen/2016/Unfallentwicklung_2015/Pressebroschuere_unfallentwicklung.pdf?__blob=publicationFile)\nNat

In [52]:
question_map = {q['id']: q['question'] for q in  documents_new}
test1 = list(map(lambda doc: {**doc, 'question': question_map.get(doc['id'], None)}, documents))
len(test1)

614

In [51]:
for doc in tqdm(documents):
    promt = get_prompt(doc)
    res = generate_questions(promt)
    id = doc["id"]
    documents_new_llama.append({"id":id, "question":res})
    

  0%|          | 0/614 [00:00<?, ?it/s]

In [53]:
with open('doc_with_q_lama.json', 'wt') as f_out:
    json.dump(test1, f_out, indent=2)

In [54]:
test1[0]

{'source': 'https://www.reddit.com/r/germany/wiki/autobahn_safety',
 'content': 'The Autobahn is a [network of interstate highways in Germany](https://en.m.wikipedia.org/wiki/Autobahn#/media/File%3AAutobahnen_in_Deutschland.svg) with a total length of more than 8,000 miles. [65%](https://en.wikipedia.org/wiki/Autobahn#Speed_limits) of the Autobahn has no speed limit. How safe can that be?\nVehicles traveled 147 billion miles on the Autobahn in 2015. 322 people died = 2.19 deaths per billion miles.\nIn the US, vehicles travelled 757 billion miles on interstate highways. 3,837 people died = 5.07 deaths per billion miles.\nThat means: If you drive on the interstate, your likelihood to die is 131% higher than for the same distance on the Autobahn.\n*sources:*\nStatistisches Bundesamt: [Unfallentwicklung auf deutschen Straßen 2015](https://www.destatis.de/DE/PresseService/Presse/Pressekonferenzen/2016/Unfallentwicklung_2015/Pressebroschuere_unfallentwicklung.pdf?__blob=publicationFile)\nNat